In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import datetime
from configparser import ConfigParser
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.metrics import davies_bouldin_score
import operator
import random
from datetime import datetime
from itertools import combinations
from statistics import mode as mode
from tqdm import tqdm 
# import import_ipynb
# import Standardize_occupation as so
import pyodbc
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.metrics import davies_bouldin_score as dbc

In [3]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [4]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [5]:
#Create dataframe using pyodbc
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# # cursor1 = conn.cursor()
# # cursor2 = conn.cursor()


In [6]:
# et_risk_df_per = pd.read_sql_query("SELECT top 2000000 * FROM [Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] with (nolock)", conn)


In [7]:
# et_risk_df_per = pd.read_sql_query("SELECT  PERSON_MASTER_ID,PEER_GROUP_ID,PAN,DIN,CKYC_NUMBER,FIRST_NAME,MIDDLE_NAME,LAST_NAME,DATE_OF_BIRTH,TRAN_AMNT,TRAN_AMNT_DEBIT,TRAN_AMNT_CREDIT,CBDT_INCOME_RANGE,PINCODE,OCCUPATION_BUSINESS  FROM [Fincore_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] with (nolock)", conn)
et_risk_df_per = pd.read_sql_query("SELECT  PERSON_MASTER_ID,CBDT_INCOME_RANGE,PEER_GROUP_AMNT,OCCUPATION_BUSINESS  FROM [Fincore_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] with (nolock)", conn)
# et_risk_df_org = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]", conn)

# et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO not in "+str(stat_rules), conn)
# et_risk_df_org_rules = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES]", conn)


In [8]:
et_risk_df_per.head()

,PERSON_MASTER_ID,CBDT_INCOME_RANGE,PEER_GROUP_AMNT,OCCUPATION_BUSINESS
0,1,nan,2264360596.00000,Government Sector
1,2,nan,566453534.00000,Government Sector
2,3,nan,565000000.00000,Government Sector
3,4,nan,566453536.00000,Government Sector
4,5,nan,566453537.00000,Government Sector


In [9]:
no_of_records = len(et_risk_df_per)

In [10]:
no_of_records

346

In [11]:
%store no_of_records

Stored 'no_of_records' (int)


In [12]:
# print(et_risk_df_per.shape,et_risk_df_org.shape)
# # print(et_risk_df_per_rules.shape,et_risk_df_org_rules.shape)

In [13]:
# print(f"There are {et_risk_df_per['OCCUPATION_BUSINESS'].nunique()} unique business in the person table")
# # print(f"There are {et_risk_df_org['OCCUPATION_BUSINESS'].nunique()} unique business in the organization table")

In [14]:
et_risk_df_per['OCCUPATION_BUSINESS'].isna().sum()
#Replacing null values in occupation with Others
et_risk_df_per['OCCUPATION_BUSINESS'].fillna(value = 'Others',inplace=True)
et_risk_df_per['OCCUPATION_BUSINESS_CLEAN'] = et_risk_df_per['OCCUPATION_BUSINESS']
# et_risk_df_per['OCCUPATION_BUSINESS_CLEAN'] = et_risk_df_per['OCCUPATION_BUSINESS'].apply(stand_occ)
# et_risk_df_org['OCCUPATION_BUSINESS'].fillna(value = 'Others',inplace=True)

#Replacing null lat and long values with 999
# et_risk_df_per['LATITUDE'].fillna(value = 999,inplace=True)
# et_risk_df_org['LATITUDE'].fillna(value = 999,inplace=True)

# et_risk_df_per['LONGITUDE'].fillna(value = 999,inplace=True)
# et_risk_df_org['LONGITUDE'].fillna(value = 999,inplace=True)

#Replacing null Tran_Amnt with 0

# et_risk_df_org['TRAN_AMNT'].fillna(value = 0,inplace=True)

In [15]:
x = et_risk_df_per['OCCUPATION_BUSINESS_CLEAN']


In [16]:
x.value_counts()

Others               145
Government Sector    113
Private Sector        53
Self Employed         18
Professional           7
Public Sector          3
Retired                3
Housewife              3
Student                1
Name: OCCUPATION_BUSINESS_CLEAN, dtype: int64

In [17]:
# All the transaction value which are null are replaced with 0

# et_risk_df_per['TRAN_AMNT'].fillna(value = 0,inplace=True)  #Remember to change
# et_risk_df_per['CBDT_INCOME_RANGE'].fillna(value = 0,inplace=True)
et_risk_df_per['PEER_GROUP_AMNT'].fillna(value = 0,inplace=True)

In [18]:
#Running the code to standardize occupation(old code)
# et_risk_df_per['OCCUPATION_BUSINESS_CLEAN'] = et_risk_df_per['OCCUPATION_BUSINESS'].apply(so.stand_occ)
# et_risk_df_org['OCCUPATION_BUSINESS_CLEAN'] = et_risk_df_org['OCCUPATION_BUSINESS'].apply(so.stand_occ)


# et_risk_df_org['OCCUPATION_BUSINESS_CLEAN'] = et_risk_df_org['OCCUPATION_BUSINESS'].apply(stand_occ)

In [19]:
# th_col_per = ['PERSON_MASTER_ID','OCCUPATION_BUSINESS_CLEAN','CBDT_INCOME_RANGE']
th_col_per = ['PERSON_MASTER_ID','OCCUPATION_BUSINESS_CLEAN','PEER_GROUP_AMNT']#Threshold Column for person(these columns are needed in the final dataframe)
# th_col_per = ['PERSON_MASTER_ID','OCCUPATION_BUSINESS_CLEAN','TRAN_AMNT']

#Uncomment to run on org
# th_col_org = ['ORGANIZATION_MASTER_ID','OCCUPATION_BUSINESS_CLEAN','TRAN_AMNT','LATITUDE','LONGITUDE','O002001001','O002001018','O002001017','O002001024', 'O002001025', 'O002001026']


In [20]:
data_ind = et_risk_df_per
#Uncomment to run on org
# data_org = et_risk_df_org

In [21]:
data_ind_U = data_ind[th_col_per]
#Uncomment to run on org
# data_org_U = data_org[th_col_org]


In [22]:
#Summary of NaN values in the person data
data_ind_U.isna().sum()

PERSON_MASTER_ID             0
OCCUPATION_BUSINESS_CLEAN    0
PEER_GROUP_AMNT              0
dtype: int64

In [23]:
# print(f"Size before dropping NaN {data_ind_U.shape}")
# # print(f"Size before dropping NaN {data_org_U.shape}")
# data_ind_U.dropna(inplace=True)
# # data_org_U.dropna(inplace=True)
# print(f"Size after dropping NaN {data_ind_U.shape}")
# # print(f"Size after dropping NaN {data_org_U.shape}")

In [24]:
data_ind_U.drop_duplicates(inplace=True)

In [25]:
no_of_records = len(data_ind_U)

In [26]:
# print(np.array(data_ind_U['TRAN_AMNT'].values).reshape(-1,1))

In [27]:
no_of_records

346

In [28]:
scaler = MinMaxScaler()
# data_ind_U['Transaction_Amount_Scaled'] = scaler.fit_transform(np.array(data_ind_U['TRAN_AMNT'].values).reshape(-1,1))
# data_ind_U['Income_Range_Scaled'] = scaler.fit_transform(np.array(data_ind_U['CBDT_INCOME_RANGE'].values).reshape(-1,1))
data_ind_U['PEER_GROUP_AMNT_Scaled'] = scaler.fit_transform(np.array(data_ind_U['PEER_GROUP_AMNT'].values).reshape(-1,1))

#Uncomment to run on org
# data_org_U['Transaction_Amount_Scaled'] = scaler.fit_transform(np.array(data_org_U['TRAN_AMNT'].values).reshape(-1,1))


In [29]:
# print(data_ind_U['Transaction_Amount_Scaled'].min(),data_ind_U['Transaction_Amount_Scaled'].max())

print(data_ind_U['PEER_GROUP_AMNT_Scaled'].min(),data_ind_U['PEER_GROUP_AMNT_Scaled'].max())
# print(data_ind_U['Income_Range_Scaled'].min(),data_ind_U['Income_Range_Scaled'].max())
#Uncomment to run on org
# print(data_org_U['Transaction_Amount_Scaled'].min(),data_org_U['Transaction_Amount_Scaled'].max())

0.0 1.0


In [30]:
#Uncomment this code when latitude and longitude comes
# data_ind_U['LATITUDE_Scaled'] = scaler.fit_transform(np.array(data_ind_U['LATITUDE'].values).reshape(-1,1))
#Uncomment to run on org
# data_org_U['LATITUDE_Scaled'] = scaler.fit_transform(np.array(data_org_U['LATITUDE'].values).reshape(-1,1))

# data_ind_U['LONGITUDE_Scaled'] = scaler.fit_transform(np.array(data_ind_U['LONGITUDE'].values).reshape(-1,1))
#Uncomment to run on org
# data_org_U['LONGITUDE_Scaled'] = scaler.fit_transform(np.array(data_org_U['LONGITUDE'].values).reshape(-1,1))


In [31]:
# data_ind_U['Location_Scaled'] = scaler.fit_transform(np.array(data_ind_U['Location_Encoded'].values).reshape(-1,1))
# data_org_U['Location_Scaled'] = scaler.fit_transform(np.array(data_org_U['Location_Encoded'].values).reshape(-1,1))


In [32]:
#Columns we are using to perform clustering
# cluster_cols = ['Transaction_Amount_Scaled','LATITUDE_Scaled','LONGITUDE_Scaled']
# cluster_cols = ['Income_Range_Scaled']
cluster_cols = ['PEER_GROUP_AMNT_Scaled']
# cluster_cols = ['Transaction_Amount_Scaled']

#Uncomment this code when geolocation comes
# cluster_cols = ['Transaction_Amount_Scaled','LATITUDE_Scaled','LONGITUDE_Scaled']

In [33]:
# #BEST FEATURE SELECTION LOGIC
# #Dictionary that stores column,number of clusters and their silhouette score
# kmin = 2
# kmax = 6
# sil_score_dict = {}
# sil_avg_list = {}
# for i in range(kmin,kmax):    
#     for j in range(2,len(cluster_cols)+1):
#         comb = list(combinations(cluster_cols,j))
#         for c in comb:
#             if len(c) > 0:
# #                 sil_score_dict[(list(c),i)] = {}
#                 cluster_labels = KMeans(i).fit_predict(df[list(c)])
#                 sil_avg = silhouette_score(df[list(c)],cluster_labels)
#                 if sil_avg !=1:
#                     sil_score_dict[c,i] = sil_avg


In [34]:
distortions_ind = {}
inertias_ind = {}

In [35]:
# class Model_Kmeans():
#     def __init__(self,df):
#         self.data = df
    

#     def cluster_data(self):

#         for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
#             num_cluster = 4
#             print(f"Clustering for profession {occ} on ",)
# #            
#             km_final = KMeans(n_clusters=4)
#             labels = km_final.fit_predict(self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,cluster_cols])
# #             print("GETTING ERROR")
#             self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,'Cluster_labels'] = labels
#         return self.data
            
# #         self.data.to_csv("/data/Synthetic_data_Clustered_"+entity_type+".csv")

In [36]:
# class Model_Kmeans():
#     def __init__(self,df):
# #         print("I have been called")
#         self.data = df
#     def silhouette_calculation(self):
# #         print("And Rohaaan will answer")
#         init_time = datetime.now()
#         n = 6
#         self.sil_score_dict = {}
#         for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
#             sil_avg_list = {}
#             print(f"Clustering for Profession {occ}")
#             distortions = {}
#             inertias = {}
#             self.sil_score_dict[occ] = {}
#             for k in tqdm(range(1,6)):
#                 time1 = datetime.now()
#                 print("Working on ",occ,k)
#                 try:
#                     X = self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,cluster_cols] #getting desired columns of that occupation
#                     print(len(X))
#                     if len(X) < k: #checking if the number of rows in X is less than k(num clusters)
#                         print("Try",occ,'Length:',len(X),k)
#                         continue
#                     cluster_labels = KMeans(n_clusters = k).fit_predict(X)
#                     sil_avg = silhouette_score(X,cluster_labels)
#                     self.sil_score_dict[occ][k] = sil_avg

#                 except ValueError:
                    
#                     sil_avg = -1 #making sil score -1 for num cluster = 1
#                     self.sil_score_dict[occ][k] = sil_avg
#                 print("Clustering done for",k)
#                 time2 = datetime.now()
#                 time3 = time2-time1
#                 print(f"It took {time3/3600} hours to cluster {occ} in {k} clusters")
#                 print(time2)
#         return self.sil_score_dict
    
#     def best_feature_selection(self):
#         kmin = 2
#         kmax = 6
#         self.sil_score_dict = {}
#         self.sil_avg_list = {}
#         self.final_dict = {}
#         for i in range(kmin,kmax):
#             for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
#                 for j in range(2,len(cluster_cols)+1):
#                     comb = list(combinations(cluster_cols,j))
#                     for c in comb:
#                         if len(c) > 0:
#             #                 sil_score_dict[(list(c),i)] = {}
#                             X = self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,list(c)]
#                             cluster_labels = KMeans(i).fit_predict(X)
#                             sil_avg = silhouette_score(X,cluster_labels)
#                             if sil_avg !=1:
#                                 sil_score_dict[c,i] = sil_avg
#                 self.final_dict[occ] = sil_score_dict               
#         return self.final_dict
                
#     def cluster_data(self):
#         sil_data = self.silhouette_calculation()
#         print("SHS",sil_data)
        
# #         self.best_feature_selection()
# #         print("SHS",self.best_feature_selection())
#         for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
#             num_cluster = max(self.sil_score_dict[occ].items(),key = operator.itemgetter(1))[0]
#             print(f"Clustering for profession {occ} on ",)
# #             tuple_res = max(self.final_dict[occ].items(), key=operator.itemgetter(1))[0]
# #             km_final = KMeans(n_clusters = tuple_res[1])
# #             labels - km_final.fit_predict(self.data.loc[self.data['OCCUPATION_BUSINESS'] == occ,cluster_cols])

#             #we will chose that k where silhouette score is maximum
#             km_final = KMeans(n_clusters=max(self.sil_score_dict[occ].items(),key = operator.itemgetter(1))[0])
#             labels = km_final.fit_predict(self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,cluster_cols])
# #             print("GETTING ERROR")
#             self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,'Cluster_labels'] = labels
#         return self.data
            
# #         self.data.to_csv("/data/Synthetic_data_Clustered_"+entity_type+".csv")

In [37]:

class Model_devies_bouldin_score():
    def __init__(self,df):
#         print("I have been called")
        self.data = df
    def silhouette_calculation(self):
#         print("And Rohaaan will answer")
        init_time = datetime.now()
        n = 6
        self.sil_score_dict = {}
        for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
            sil_avg_list = {}
            print(f"Clustering for Profession {occ}")
            distortions = {}
            inertias = {}
            self.sil_score_dict[occ] = {}
            for k in tqdm(range(1,6)):
                time1 = datetime.now()
                print("Working on ",occ,k)
                try:
                    X = self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,cluster_cols] #getting desired columns of that occupation
                    print("lenX",len(X))
                    if len(X) < k: #checking if the number of rows in X is less than k(num clusters)
                        print("Try",occ,'Length:',len(X),k)
                        continue
                    cluster_labels = KMeans(n_clusters = k).fit_predict(X)
                    sil_avg = dbc(X,cluster_labels)
                    self.sil_score_dict[occ][k] = sil_avg

                except ValueError:
                    print("Value Error")
                    sil_avg = 1 #making sil score -1 for num cluster = 1
                    self.sil_score_dict[occ][k] = sil_avg
                print("Clustering done for",k)
                time2 = datetime.now()
                time3 = time2-time1
                print(f"It took {time3/3600} hours to cluster {occ} in {k} clusters")
                print(time2)
        return self.sil_score_dict
    
    def best_feature_selection(self):
        kmin = 2
        kmax = 6
        self.sil_score_dict = {}
        self.sil_avg_list = {}
        self.final_dict = {}
        for i in range(kmin,kmax):
            for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
                for j in range(2,len(cluster_cols)+1):
                    comb = list(combinations(cluster_cols,j))
                    for c in comb:
                        if len(c) > 0:
            #                 sil_score_dict[(list(c),i)] = {}
                            X = self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,list(c)]
                            cluster_labels = KMeans(i).fit_predict(X)
                            sil_avg = silhouette_score(X,cluster_labels)
                            if sil_avg !=1:
                                sil_score_dict[c,i] = sil_avg
                self.final_dict[occ] = sil_score_dict               
        return self.final_dict
                
    def cluster_data(self):
        sil_data = self.silhouette_calculation()
        print("SHS",sil_data)
        
#         self.best_feature_selection()
#         print("SHS",self.best_feature_selection())
        for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
            num_cluster = min(self.sil_score_dict[occ].items(),key = operator.itemgetter(1))[0]
            print(f"Clustering for profession {occ} on ",)
#             tuple_res = max(self.final_dict[occ].items(), key=operator.itemgetter(1))[0]
#             km_final = KMeans(n_clusters = tuple_res[1])
#             labels - km_final.fit_predict(self.data.loc[self.data['OCCUPATION_BUSINESS'] == occ,cluster_cols])

            #we will chose that k where silhouette score is maximum
            km_final = KMeans(n_clusters=min(self.sil_score_dict[occ].items(),key = operator.itemgetter(1))[0])
            labels = km_final.fit_predict(self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,cluster_cols])
#             print("GETTING ERROR")
            self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,'Cluster_labels'] = labels
        return self.data
            
#         self.data.to_csv("/data/Synthetic_data_Clustered_"+entity_type+".csv")

In [38]:
# class Model_Kmeans():
#     def __init__(self,df):
# #         print("I have been called")
#         self.data = df
    
#     def silhouette_calculation(self):
#         self.sil_score_dict = {'Others': {1: -1, 2: -1, 3: -1, 4: -1, 5: -1}, 'Self Employed': {1: -1, 2: 0.9999593920358186, 3: 0.9999081074672636, 4: 0.9999114318550805, 5: 0.9994774670267123}, 'Government': {1: -1, 2: 0.9999858485218839, 3: 0.9999099000100459, 4: 0.9997469740690581, 5: 0.9982632105869318}, 'Business': {1: -1, 2: 0.999912764736669, 3: 0.999474159612679, 4: 0.998846602493587, 5: 0.998503250220317}, 'Not Declared': {1: -1, 2: 0.9994937746979228, 3: 0.9993869744959869, 4: 0.9970983438735268, 5: 0.9924803684859484}, 'Average Salaried': {1: -1, 2: 0.9999732584031886, 3: 0.999861685876604, 4: 0.9998618856043362, 5: 0.9996248244645971}, 'Housewife': {1: -1, 2: 0.9995854983478283, 3: 0.999103040610443, 4: 0.9980953473904257, 5: 0.9980955170271096}, 'Metal Business': {1: -1, 2: 0.9958263268965685, 3: 0.9786991738910085, 4: 0.9605213395065433, 5: 0.8393343279109037}, 'Doctor': {1: -1, 2: 0.9975240560232588, 3: 0.9914239802375339, 4: 0.7596732126987716, 5: 0.7585051360197337}, 'Agriculture': {1: -1, 2: 0.9991920064605758, 3: 0.9950297355530854, 4: 0.9950282799074055, 5: 0.9023982691786087}, 'Engineer': {1: -1, 2: 0.9934046106934756, 3: 0.8469890666623305, 4: 0.7448565341193297, 5: 0.7457902755941127}, 'Retired': {1: -1, 2: 0.9995601797287804, 3: 0.9994206016867495, 4: 0.997200828500462, 5: 0.993016252717963}, 'Student': {1: -1, 2: 0.9999892176143073, 3: 0.9999722658100986, 4: 0.999835482873445, 5: 0.999026286980271}, 'High Salaried': {1: -1, 2: 0.9999189251100865, 3: 0.9995699790680087, 4: 0.9981583015983165, 5: 0.9960161714215744}, 'Property Dealings': {1: -1, 2: 0.9999527654591881, 3: 0.9993250632194768, 4: 0.9974906328187055, 5: 0.997486177237193}, 'Helper': {1: -1, 2: 0.9983066627337424, 3: 0.9954451486496709, 4: 0.9954058900207182, 5: 0.8907002775599244}, 'Store Owner': {1: -1, 2: 0.9967168439707761, 3: 0.9204890628062787, 4: 0.8967999295265043, 5: 0.7484671691507707}, 'Manufacturer': {1: -1, 2: 0.9996550334703684, 3: 0.9978058225796215, 4: 0.9972326932482567, 5: 0.988892474008283}, 'Stock Trading': {1: -1, 2: 0.9991205970076172, 3: 0.9995758811769558, 4: 0.9987944936692964, 5: 0.99626681044861}, 'Gas Station': {1: -1, 2: 0.9948628503731004, 3: 0.7838871330977213, 4: 0.7063686314008618, 5: 0.7070008579183414}, 'Teacher': {1: -1, 2: 0.9993073934139635, 3: 0.9931758743103802, 4: 0.9791507122254794, 5: 0.9791997810194513}, 'Jeweller': {1: -1, 2: 0.9944775593742973, 3: 0.9940746701111378, 4: 0.9917108921152105, 5: 0.9851004747842613}, 'Manager': {1: -1, 2: 0.9995846189879848, 3: 0.997092852338378, 4: 0.9932910624326226, 5: 0.9931085979238667}, 'Contractor': {1: -1, 2: 0.854665858209764, 3: 0.8516212335452136, 4: 0.7423269718508302, 5: 0.7438299124286537}, 'Retailer': {1: -1, 2: 0.9986657505605661, 3: 0.9900409558170957, 4: 0.8732761152510514, 5: 0.8650810352759549}, 'Accounts and Finance': {1: -1, 2: 0.9997872406171219, 3: 0.9977663796508954, 4: 0.9758491592897138, 5: 0.9757071216616988}, 'Lawyer': {1: -1, 2: 0.9998590551059309, 3: 0.9989194279368487, 4: 0.8538687859476395, 5: 0.8509390461388446}, 'CEO': {1: -1, 2: 0.9977044915575652, 3: 0.9947337678706236, 4: 0.991101911772474, 5: 0.8636494302819995}, 'Mining': {1: -1, 2: 0.9898582418908489, 3: 0.8753521587107874, 4: 0.8285625606832285, 5: 0.7139413203615163}, 'Actor/Artist': {1: -1, 2: 0.991179448932345, 3: 0.8605895112938831, 4: 0.8547840473232179, 5: 0.7451095498927909}, 'Commision Agent': {1: -1, 2: 0.9974301761582647, 3: 0.9963375809164849, 4: 0.991868850324092, 5: 0.9771018244214416}, 'Director': {1: -1, 2: 0.9990312371813991, 3: 0.9986201325491484, 4: 0.9983140044995152, 5: 0.9945707102020963}, 'Trustee': {1: -1, 2: 0.9965864335007005, 3: 0.9912342023087024, 4: 0.9913280196111037, 5: 0.9912543650003244}, 'Consultant': {1: -1, 2: 0.9974199132274957, 3: 0.9882604561267236, 4: 0.8378740886686522, 5: 0.7543303473814116}, 'Secretary': {1: -1, 2: 0.8891848015799844, 3: 0.8674450383830701, 4: 0.7444219266025037, 5: 0.7425028818899574}, 'Import-Export': {1: -1, 2: 0.9992889452000588, 3: 0.9988811055491394, 4: 0.9971032444215038, 5: 0.9931261498813067}, 'Money Changer': {1: -1, 2: 0.9706098604813469, 3: 0.8147377028805535, 4: 0.8087377108669346, 5: 0.8251917324834793}, 'Politician': {1: -1, 2: 0.9896038022009277, 3: 0.9852148725656451, 4: 0.7994662057406815, 5: 0.7203955924839752}, 'Alcohol Business': {1: -1, 2: 0.9868118679065201, 3: 0.7835861374306422, 4: 0.7408611365951875, 5: 0.7229055195159397}, 'Tour & Travel': {1: -1, 2: 0.9951936772344347, 3: 0.9903697685904355, 4: 0.9745393804419599, 5: 0.9137041044349832}, 'Tobacco': {1: -1, 2: 0.9955327363981464, 3: 0.8542572195368762, 4: 0.738894704634517, 5: 0.7454878512485418}, 'Investment': {1: -1, 2: 0.9943961956513977, 3: 0.9845142875371372, 4: 0.8347279465542478, 5: 0.7372262992941542}, 'Auditing': {1: -1, 2: 0.969536531383986, 3: 0.8185405586439257, 4: 0.7639530786935524, 5: 0.81310298960249}, 'Chitfund': {1: -1, 2: 0.8834486579101468, 3: 0.7983138773204599, 4: 0.7663192992944874, 5: 0.8406346179224666}, 'Media/Entertainment': {1: -1, 2: 0.9777503555941491, 3: 0.8567743728965393, 4: 0.7364883352717038, 5: 0.7361659095331903}, 'Unemployed': {1: -1, 2: 0.9770616858061453, 3: 0.8626345576168749, 4: 0.8010578905766006, 5: 0.7973491228584628}, 'Cement Business': {1: -1, 2: 0.9904147421616074, 3: 0.8577356927525285, 4: 0.729775047636267, 5: 0.7338190549659772}, 'Bullion Dealer': {1: -1, 2: 0.9738856638613536, 3: 0.9614853927996325, 4: 0.7848619339224836, 5: 0.697089872055426}, 'Clerk': {1: -1, 2: 0.9056210640539609, 3: 0.7661498023675626, 4: 0.7861436144432657, 5: 0.8386360714546925}, 'Freelancing': {1: -1, 2: 0.7263948848450875, 3: 0.8033327252622381, 4: 0.8222879881348403, 5: 0.9548764700175146}}
# # #         print("And Rohaaan will answer")
# #         init_time = datetime.now()
# #         n = 6
# #         self.sil_score_dict = {}
# #         for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
# #             sil_avg_list = {}
# #             print(f"Clustering for Profession {occ}")
# #             distortions = {}
# #             inertias = {}
# #             self.sil_score_dict[occ] = {}
# #             for k in range(1,6):
# #                 time1 = datetime.now()
# #                 print("Working on ",occ,k)
# #                 try:
# #                     X = self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,cluster_cols] #getting desired columns of that occupation
# #                     print(len(X))
# #                     if len(X) < k: #checking if the number of rows in X is less than k(num clusters)
# #                         print("Try",occ,'Length:',len(X),k)
# #                         continue
# #                     cluster_labels = KMeans(n_clusters = k).fit_predict(X)
# #                     sil_avg = silhouette_score(X,cluster_labels)
# #                     self.sil_score_dict[occ][k] = sil_avg

# #                 except ValueError:
                    
# #                     sil_avg = -1 #making sil score -1 for num cluster = 1
# #                     self.sil_score_dict[occ][k] = sil_avg
# #                 print("Clustering done for",k)
# #                 time2 = datetime.now()
# #                 time3 = time2-time1
# #                 print(f"It took {time3/3600} hours to cluster {occ} in {k} clusters")
#         return self.sil_score_dict
    
# #     def best_feature_selection(self):
# #         kmin = 2
# #         kmax = 6
# #         self.sil_score_dict = {}
# #         self.sil_avg_list = {}
# #         self.final_dict = {}
# #         for i in range(kmin,kmax):
# #             for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
# #                 for j in range(2,len(cluster_cols)+1):
# #                     comb = list(combinations(cluster_cols,j))
# #                     for c in comb:
# #                         if len(c) > 0:
# #             #                 sil_score_dict[(list(c),i)] = {}
# #                             X = self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,list(c)]
# #                             cluster_labels = KMeans(i).fit_predict(X)
# #                             sil_avg = silhouette_score(X,cluster_labels)
# #                             if sil_avg !=1:
# #                                 sil_score_dict[c,i] = sil_avg
# #                 self.final_dict[occ] = sil_score_dict               
# #         return self.final_dict
                
#     def cluster_data(self):
#         sil_data = self.silhouette_calculation()
# #         sil_score_dict = {'Others': {1: -1, 2: -1, 3: -1, 4: -1, 5: -1}, 'Self Employed': {1: -1, 2: 0.9999593920358186, 3: 0.9999081074672636, 4: 0.9999114318550805, 5: 0.9994774670267123}, 'Government': {1: -1, 2: 0.9999858485218839, 3: 0.9999099000100459, 4: 0.9997469740690581, 5: 0.9982632105869318}, 'Business': {1: -1, 2: 0.999912764736669, 3: 0.999474159612679, 4: 0.998846602493587, 5: 0.998503250220317}, 'Not Declared': {1: -1, 2: 0.9994937746979228, 3: 0.9993869744959869, 4: 0.9970983438735268, 5: 0.9924803684859484}, 'Average Salaried': {1: -1, 2: 0.9999732584031886, 3: 0.999861685876604, 4: 0.9998618856043362, 5: 0.9996248244645971}, 'Housewife': {1: -1, 2: 0.9995854983478283, 3: 0.999103040610443, 4: 0.9980953473904257, 5: 0.9980955170271096}, 'Metal Business': {1: -1, 2: 0.9958263268965685, 3: 0.9786991738910085, 4: 0.9605213395065433, 5: 0.8393343279109037}, 'Doctor': {1: -1, 2: 0.9975240560232588, 3: 0.9914239802375339, 4: 0.7596732126987716, 5: 0.7585051360197337}, 'Agriculture': {1: -1, 2: 0.9991920064605758, 3: 0.9950297355530854, 4: 0.9950282799074055, 5: 0.9023982691786087}, 'Engineer': {1: -1, 2: 0.9934046106934756, 3: 0.8469890666623305, 4: 0.7448565341193297, 5: 0.7457902755941127}, 'Retired': {1: -1, 2: 0.9995601797287804, 3: 0.9994206016867495, 4: 0.997200828500462, 5: 0.993016252717963}, 'Student': {1: -1, 2: 0.9999892176143073, 3: 0.9999722658100986, 4: 0.999835482873445, 5: 0.999026286980271}, 'High Salaried': {1: -1, 2: 0.9999189251100865, 3: 0.9995699790680087, 4: 0.9981583015983165, 5: 0.9960161714215744}, 'Property Dealings': {1: -1, 2: 0.9999527654591881, 3: 0.9993250632194768, 4: 0.9974906328187055, 5: 0.997486177237193}, 'Helper': {1: -1, 2: 0.9983066627337424, 3: 0.9954451486496709, 4: 0.9954058900207182, 5: 0.8907002775599244}, 'Store Owner': {1: -1, 2: 0.9967168439707761, 3: 0.9204890628062787, 4: 0.8967999295265043, 5: 0.7484671691507707}, 'Manufacturer': {1: -1, 2: 0.9996550334703684, 3: 0.9978058225796215, 4: 0.9972326932482567, 5: 0.988892474008283}, 'Stock Trading': {1: -1, 2: 0.9991205970076172, 3: 0.9995758811769558, 4: 0.9987944936692964, 5: 0.99626681044861}, 'Gas Station': {1: -1, 2: 0.9948628503731004, 3: 0.7838871330977213, 4: 0.7063686314008618, 5: 0.7070008579183414}, 'Teacher': {1: -1, 2: 0.9993073934139635, 3: 0.9931758743103802, 4: 0.9791507122254794, 5: 0.9791997810194513}, 'Jeweller': {1: -1, 2: 0.9944775593742973, 3: 0.9940746701111378, 4: 0.9917108921152105, 5: 0.9851004747842613}, 'Manager': {1: -1, 2: 0.9995846189879848, 3: 0.997092852338378, 4: 0.9932910624326226, 5: 0.9931085979238667}, 'Contractor': {1: -1, 2: 0.854665858209764, 3: 0.8516212335452136, 4: 0.7423269718508302, 5: 0.7438299124286537}, 'Retailer': {1: -1, 2: 0.9986657505605661, 3: 0.9900409558170957, 4: 0.8732761152510514, 5: 0.8650810352759549}, 'Accounts and Finance': {1: -1, 2: 0.9997872406171219, 3: 0.9977663796508954, 4: 0.9758491592897138, 5: 0.9757071216616988}, 'Lawyer': {1: -1, 2: 0.9998590551059309, 3: 0.9989194279368487, 4: 0.8538687859476395, 5: 0.8509390461388446}, 'CEO': {1: -1, 2: 0.9977044915575652, 3: 0.9947337678706236, 4: 0.991101911772474, 5: 0.8636494302819995}, 'Mining': {1: -1, 2: 0.9898582418908489, 3: 0.8753521587107874, 4: 0.8285625606832285, 5: 0.7139413203615163}, 'Actor/Artist': {1: -1, 2: 0.991179448932345, 3: 0.8605895112938831, 4: 0.8547840473232179, 5: 0.7451095498927909}, 'Commision Agent': {1: -1, 2: 0.9974301761582647, 3: 0.9963375809164849, 4: 0.991868850324092, 5: 0.9771018244214416}, 'Director': {1: -1, 2: 0.9990312371813991, 3: 0.9986201325491484, 4: 0.9983140044995152, 5: 0.9945707102020963}, 'Trustee': {1: -1, 2: 0.9965864335007005, 3: 0.9912342023087024, 4: 0.9913280196111037, 5: 0.9912543650003244}, 'Consultant': {1: -1, 2: 0.9974199132274957, 3: 0.9882604561267236, 4: 0.8378740886686522, 5: 0.7543303473814116}, 'Secretary': {1: -1, 2: 0.8891848015799844, 3: 0.8674450383830701, 4: 0.7444219266025037, 5: 0.7425028818899574}, 'Import-Export': {1: -1, 2: 0.9992889452000588, 3: 0.9988811055491394, 4: 0.9971032444215038, 5: 0.9931261498813067}, 'Money Changer': {1: -1, 2: 0.9706098604813469, 3: 0.8147377028805535, 4: 0.8087377108669346, 5: 0.8251917324834793}, 'Politician': {1: -1, 2: 0.9896038022009277, 3: 0.9852148725656451, 4: 0.7994662057406815, 5: 0.7203955924839752}, 'Alcohol Business': {1: -1, 2: 0.9868118679065201, 3: 0.7835861374306422, 4: 0.7408611365951875, 5: 0.7229055195159397}, 'Tour & Travel': {1: -1, 2: 0.9951936772344347, 3: 0.9903697685904355, 4: 0.9745393804419599, 5: 0.9137041044349832}, 'Tobacco': {1: -1, 2: 0.9955327363981464, 3: 0.8542572195368762, 4: 0.738894704634517, 5: 0.7454878512485418}, 'Investment': {1: -1, 2: 0.9943961956513977, 3: 0.9845142875371372, 4: 0.8347279465542478, 5: 0.7372262992941542}, 'Auditing': {1: -1, 2: 0.969536531383986, 3: 0.8185405586439257, 4: 0.7639530786935524, 5: 0.81310298960249}, 'Chitfund': {1: -1, 2: 0.8834486579101468, 3: 0.7983138773204599, 4: 0.7663192992944874, 5: 0.8406346179224666}, 'Media/Entertainment': {1: -1, 2: 0.9777503555941491, 3: 0.8567743728965393, 4: 0.7364883352717038, 5: 0.7361659095331903}, 'Unemployed': {1: -1, 2: 0.9770616858061453, 3: 0.8626345576168749, 4: 0.8010578905766006, 5: 0.7973491228584628}, 'Cement Business': {1: -1, 2: 0.9904147421616074, 3: 0.8577356927525285, 4: 0.729775047636267, 5: 0.7338190549659772}, 'Bullion Dealer': {1: -1, 2: 0.9738856638613536, 3: 0.9614853927996325, 4: 0.7848619339224836, 5: 0.697089872055426}, 'Clerk': {1: -1, 2: 0.9056210640539609, 3: 0.7661498023675626, 4: 0.7861436144432657, 5: 0.8386360714546925}, 'Freelancing': {1: -1, 2: 0.7263948848450875, 3: 0.8033327252622381, 4: 0.8222879881348403, 5: 0.9548764700175146}}
#         print("SHS",sil_data)
        
# #         self.best_feature_selection()
# #         print("SHS",self.best_feature_selection())
#         for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
#             num_cluster = max(self.sil_score_dict[occ].items(),key = operator.itemgetter(1))[0]
#             print(f"Clustering for profession {occ} on ",)
# #             tuple_res = max(self.final_dict[occ].items(), key=operator.itemgetter(1))[0]
# #             km_final = KMeans(n_clusters = tuple_res[1])
# #             labels - km_final.fit_predict(self.data.loc[self.data['OCCUPATION_BUSINESS'] == occ,cluster_cols])

#             #we will chose that k where silhouette score is maximum
#             km_final = KMeans(n_clusters=max(self.sil_score_dict[occ].items(),key = operator.itemgetter(1))[0])
#             labels = km_final.fit_predict(self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,cluster_cols])
# #             print("GETTING ERROR")
#             self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ,'Cluster_labels'] = labels
#         return self.data
            
# #         self.data.to_csv("/data/Synthetic_data_Clustered_"+entity_type+".csv")

In [39]:
print(datetime.now())

2023-02-21 14:12:45.424295


In [40]:
# ind_U = Model_Kmeans(data_ind_U)
# ind_data_U = ind_U.cluster_data()
# print()
# print("*** Starting for Organization ***")
# print()
# #Uncomment to run on org          
# # org_U = Model_Kmeans(data_org_U)
# # org_data_U = org_U.cluster_data()


In [41]:
print(datetime.now())

2023-02-21 14:12:45.430250


In [42]:
data_ind_U[data_ind_U['PERSON_MASTER_ID'] == 229]

,PERSON_MASTER_ID,OCCUPATION_BUSINESS_CLEAN,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled
177,229,Others,4767947886750.50000,1.00000


In [43]:
ind_U = Model_devies_bouldin_score(data_ind_U)
ind_data_U = ind_U.cluster_data()

Clustering for Profession Government Sector


  0%|          | 0/5 [00:00<?, ?it/s]

Working on  Government Sector 1
lenX 113
Value Error
Clustering done for 1
It took 0:00:00.000003 hours to cluster Government Sector in 1 clusters
2023-02-21 14:12:45.453664
Working on  Government Sector 2
lenX 113
Clustering done for 2
It took 0:00:00.000004 hours to cluster Government Sector in 2 clusters
2023-02-21 14:12:45.467586
Working on  Government Sector 3
lenX 113
Clustering done for 3
It took 0:00:00.000004 hours to cluster Government Sector in 3 clusters
2023-02-21 14:12:45.483682
Working on  Government Sector 4
lenX 113
Clustering done for 4
It took 0:00:00.000005 hours to cluster Government Sector in 4 clusters
2023-02-21 14:12:45.501940
Working on  Government Sector 5
lenX 113


100%|██████████| 5/5 [00:00<00:00, 63.13it/s]


Clustering done for 5
It took 0:00:00.000006 hours to cluster Government Sector in 5 clusters
2023-02-21 14:12:45.522807
Clustering for Profession Self Employed


...

Value Error
Clustering done for 2
It took 0:00:00.000129 hours to cluster Housewife in 2 clusters
2023-02-21 14:12:46.393150
Working on  Housewife 3
lenX 3


100%|██████████| 5/5 [00:00<00:00,  5.32it/s]


Value Error
Clustering done for 3
It took 0:00:00.000129 hours to cluster Housewife in 3 clusters
2023-02-21 14:12:46.857576
Working on  Housewife 4
lenX 3
Try Housewife Length: 3 4
Working on  Housewife 5
lenX 3
Try Housewife Length: 3 5
Clustering for Profession Retired


100%|██████████| 5/5 [00:00<00:00, 134.23it/s]

Working on  Retired 1
lenX 3
Value Error
Clustering done for 1
It took 0:00:00.000003 hours to cluster Retired in 1 clusters
2023-02-21 14:12:46.873591
Working on  Retired 2
lenX 3
Clustering done for 2
It took 0:00:00.000003 hours to cluster Retired in 2 clusters
2023-02-21 14:12:46.884033
Working on  Retired 3
lenX 3
Clustering done for 3
It took 0:00:00.000004 hours to cluster Retired in 3 clusters
2023-02-21 14:12:46.898076
Working on  Retired 4
lenX 3
Try Retired Length: 3 4
Working on  Retired 5
lenX 3
Try Retired Length: 3 5
SHS {'Government Sector': {1: 1, 2: 0.44278098561806123, 3: 0.3742194389115319, 4: 0.15691172937125797, 5: 0.12611857213292738}, 'Self Employed': {1: 1, 2: 0.09651837706136346, 3: 0.17807900936723098, 4: 0.09023270836778849, 5: 0.07360922189740725}, 'Others': {1: 1, 2: 0.004757966393417318, 3: 0.03480417890044226, 4: 0.0005593028169293037, 5: 0.0008180941368600321}, 'Private Sector': {1: 1, 2: 0.0007608155790375941, 3: 0.2157457051920201, 4: 0.24669912613440

In [44]:
print(datetime.now())

2023-02-21 14:12:47.008549


In [45]:
ind_data_U['Cluster_labels'] = ind_data_U['Cluster_labels'].apply(lambda x:int(x))
#Uncomment to run on org
# org_data_U['Cluster_labels'] = org_data_U['Cluster_labels'].apply(lambda x:int(x))
# ind_data_U.to_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\Synthetic_data_Clustered_individual_U.csv",index=False)
#Uncomment to run on org
# org_data_U.to_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\Synthetic_data_Clustered_organization_U.csv",index=False)


In [46]:
ind_data_U['PEER_GROUP_ID'] = ind_data_U.apply(lambda row:row['OCCUPATION_BUSINESS_CLEAN']+"_"+str(row['Cluster_labels']),axis=1)

In [47]:
y= ind_data_U['PEER_GROUP_ID'].value_counts().reset_index()

In [48]:
binning_data = ind_data_U[ind_data_U['OCCUPATION_BUSINESS_CLEAN']=='Others']

In [49]:
binning_data.head()

,PERSON_MASTER_ID,OCCUPATION_BUSINESS_CLEAN,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,Cluster_labels,PEER_GROUP_ID
9,10,Others,0.00000,0.00000,0,Others_0
35,46,Others,0.00000,0.00000,0,Others_0
36,47,Others,0.00000,0.00000,0,Others_0
37,48,Others,775997555028.00000,0.16275,2,Others_2
38,49,Others,0.00000,0.00000,0,Others_0


In [50]:
bins = [0,1,500000,1000000,2000000,3000000,4000000,5000000,7500000,10000000]
labels = ['Others_0','Others_1','Others_2','Others_3','Others_4','Others_5','Others_6','Others_7','Others_8']
cluster_labels = [0,1,2,3,4,5,6,7,8]

In [51]:
binning_data['PEER_GROUP_ID'] = pd.cut(x = binning_data['PEER_GROUP_AMNT'], bins = bins, labels = labels, include_lowest = True)
binning_data['Cluster_labels'] = pd.cut(x = binning_data['PEER_GROUP_AMNT'], bins = bins, labels = cluster_labels, include_lowest = True)


In [52]:
binning_data['PEER_GROUP_ID'] = binning_data['PEER_GROUP_ID'].astype('str')
binning_data['Cluster_labels'] = binning_data['Cluster_labels'].astype('float64')

In [53]:
binning_data['PEER_GROUP_ID'].replace('nan','Others_9',inplace =  True)
binning_data['Cluster_labels'].fillna(9,inplace = True)

In [54]:
binning_data[binning_data['PERSON_MASTER_ID']==229]

,PERSON_MASTER_ID,OCCUPATION_BUSINESS_CLEAN,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,Cluster_labels,PEER_GROUP_ID
177,229,Others,4767947886750.50000,1.00000,9.00000,Others_9


In [55]:
binning_data['PEER_GROUP_ID'] = binning_data['PEER_GROUP_ID'].astype('str')
binning_data['Cluster_labels'] = binning_data['Cluster_labels'].astype('int32')

In [56]:
binning_data = binning_data[['PERSON_MASTER_ID','PEER_GROUP_ID','Cluster_labels']]

In [57]:
binning_data['PEER_GROUP_ID'].value_counts()

Others_0    113
Others_2     12
Others_8     10
Others_9      7
Others_5      2
Others_1      1
Name: PEER_GROUP_ID, dtype: int64

In [58]:
ind_data_U.head()

,PERSON_MASTER_ID,OCCUPATION_BUSINESS_CLEAN,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,Cluster_labels,PEER_GROUP_ID
0,1,Government Sector,2264360596.00000,0.00047,0,Government Sector_0
1,2,Government Sector,566453534.00000,0.00012,0,Government Sector_0
2,3,Government Sector,565000000.00000,0.00012,0,Government Sector_0
3,4,Government Sector,566453536.00000,0.00012,0,Government Sector_0
4,5,Government Sector,566453537.00000,0.00012,0,Government Sector_0


In [59]:
ind_data_U = pd.merge(ind_data_U,binning_data,on='PERSON_MASTER_ID',how = 'left')

In [60]:
ind_data_U['PEER_GROUP_ID'] = ind_data_U[['PEER_GROUP_ID_y','PEER_GROUP_ID_x']].bfill(axis=1).iloc[:, 0]


In [61]:
ind_data_U['Cluster_labels_y'] = ind_data_U['Cluster_labels_y'].astype('float64')

In [62]:
ind_data_U['Cluster_labels'] = ind_data_U[['Cluster_labels_y','Cluster_labels_x']].bfill(axis=1).iloc[:, 0]

In [63]:
ind_data_U['Cluster_labels'] = ind_data_U['Cluster_labels'].astype('int32')

In [64]:
ind_data_U.drop(columns= ['Cluster_labels_x','PEER_GROUP_ID_x','PEER_GROUP_ID_y','Cluster_labels_y'],inplace = True)

In [65]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [66]:
from tqdm import tqdm

In [67]:
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()
# tmp = cursor.execute("SELECT * FROM fincore_analytics.RISK_PERSON_SUMMARY_STG ;").fetchone()
count = 1
#Updating peer_group_id in original table where person_master_id is matching
for ind,row in tqdm(ind_data_U.iterrows()):
    if count%10000 == 0:
        print(count," VALUES DONE")
    count+=1
    cursor.execute("""update fincore_db.fincore_analytics.RISK_PERSON_SUMMARY_STG 
    set PEER_GROUP_ID = '{}'
    where PERSON_MASTER_ID = {} """.format(row['PEER_GROUP_ID'], row['PERSON_MASTER_ID']))
    
# count = 0
# for ind,row in data_org_U.iterrows():
#     if count%10000 == 0:
#         print(count," VALUES DONE")
#     count+=1
#     cursor.execute("""update fincore_analytics.RISK_ORGANIZATION_SUMMARY_STG 
#     set RISK_ORGANIZATION_SUMMARY_STG.PEER_GROUP_ID = ?
#     where RISK_ORGANIZATION_SUMMARY_STG.ORGANIZATION_MASTER_ID = ?""",(row['PEER_GROUP_ID'], row['ORGANIZATION_MASTER_ID']))
    
        

conn.commit()
conn.close()

346it [00:00, 1002.73it/s]


In [68]:
ind_data_U.to_csv(r"/data/peer_updated.csv",index = False)

In [69]:
%store no_of_records

Stored 'no_of_records' (int)


In [70]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [71]:
# # pyodbc.drivers()
# # # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# df = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] with ", conn)
# df1 = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES]", conn)
# # conn.close()


In [72]:
# x = df[df['PEER_GROUP_ID'] == 'Government_0']['PERSON_MASTER_ID']
# x= pd.DataFrame(x,columns = ['PERSON_MASTER_ID'])
# # print(x)
# df2 = x.merge(df1,how = 'inner',on ='PERSON_MASTER_ID' )

# y = df2[df2['RULE_NO'] == 'P002001009']['RULE_VALUE'].astype('int64')

# print(y.describe())
# print("median = ",np.median(y))
# print(mode(y))


In [73]:
ind_data_U = pd.read_csv(r"/data/peer_updated.csv")

In [74]:
ind_data_U

,PERSON_MASTER_ID,OCCUPATION_BUSINESS_CLEAN,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,PEER_GROUP_ID,Cluster_labels
0,1,Government Sector,2264360596.00000,0.00047,Government Sector_0,0
1,2,Government Sector,566453534.00000,0.00012,Government Sector_0,0
2,3,Government Sector,565000000.00000,0.00012,Government Sector_0,0
3,4,Government Sector,566453536.00000,0.00012,Government Sector_0,0
4,5,Government Sector,566453537.00000,0.00012,Government Sector_0,0
...,...,...,...,...,...,...
341,447,Private Sector,30000025.00000,0.00001,Private Sector_0,0
342,448,Private Sector,0.00000,0.00000,Private Sector_0,0
343,449,Private Sector,1200000.00000,0.00000,Private Sector_0,0
344,450,Private Sector,1200000.00000,0.00000,Private Sector_0,0


In [75]:
x = ind_data_U.groupby('PEER_GROUP_ID').count()

In [76]:
x ['COUNTS'] = x['PERSON_MASTER_ID'] 

In [77]:
# x.drop(columns = ['OCCUPATION_BUSINESS_CLEAN','TRAN_AMNT','PERSON_MASTER_ID','CBDT_INCOME_RANGE','Transaction_Amount_Scaled','Income_Range_Scaled','Income_Range_Scaled'],inplace = True)
x.drop(columns = ['OCCUPATION_BUSINESS_CLEAN','PERSON_MASTER_ID','PEER_GROUP_AMNT','PEER_GROUP_AMNT_Scaled','Cluster_labels'],inplace = True)

In [78]:
x.reset_index(inplace = True)

In [79]:
# x.to_csv('results.csv')

In [80]:
peer_chg = x[x['COUNTS'] < 30]

In [81]:
peer_chg.reset_index(inplace = True)

In [82]:
# x1 = ind_data_U.merge(peer_chg,how='inner',on='PEER_GROUP_ID')

In [83]:
# x1  = x1['PERSON_MASTER_ID'].to_list()

In [84]:
# x1

In [85]:
peer_chg = peer_chg['PEER_GROUP_ID'].to_list()

In [86]:
peer_chg

['Government Sector_1',
 'Government Sector_2',
 'Government Sector_3',
 'Government Sector_4',
 'Housewife_0',
 'Others_1',
 'Others_2',
 'Others_5',
 'Others_8',
 'Others_9',
 'Private Sector_1',
 'Professional_0',
 'Professional_1',
 'Public Sector_0',
 'Public Sector_1',
 'Retired_0',
 'Retired_1',
 'Self Employed_0',
 'Self Employed_1',
 'Self Employed_2',
 'Self Employed_3',
 'Self Employed_4',
 'Student_0']

In [87]:
# peer_chg = tuple(peer_chg)

In [88]:
# from tqdm import tqdm

In [89]:
# for ind,row in tqdm(ind_data_U.iterrows()):
#     for peer in peer_chg:
#         if (row['PEER_GROUP_ID']== peer):
# #             print(row['PEER_GROUP_ID'],peer)
# #             break
#             ind_data_U['PEER_GROUP_ID'][ind] = 'Not_available' 
# #             row['PEER_GROUP_ID'] = 'Not_available'

    

In [90]:
# z = ind_data_U[ind_data_U['PEER_GROUP_ID'] == 'Accounts and Finance_0'].head(10)

In [91]:
# z.to_csv('results2.csv')

In [92]:
ind_data_U_temp = ind_data_U[ind_data_U['PEER_GROUP_ID'].isin(peer_chg)]

In [93]:
ind_data_U_temp

,PERSON_MASTER_ID,OCCUPATION_BUSINESS_CLEAN,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,PEER_GROUP_ID,Cluster_labels
5,6,Self Employed,0.00000,0.00000,Self Employed_0,0
6,7,Self Employed,15000000.00000,0.00000,Self Employed_3,3
32,43,Government Sector,1034801000000.00000,0.21703,Government Sector_4,4
33,44,Government Sector,1034800000000.00000,0.21703,Government Sector_4,4
37,48,Others,775997555028.00000,0.16275,Others_9,9
...,...,...,...,...,...,...
334,419,Others,10000000.00000,0.00000,Others_8,8
335,420,Self Employed,10000000.00000,0.00000,Self Employed_2,2
336,421,Others,10000000.00000,0.00000,Others_8,8
337,422,Self Employed,10000000.00000,0.00000,Self Employed_2,2


In [94]:
peer_chg = tuple(peer_chg)

In [95]:
# peer_grp_1 = pd.read_sql_query("select PEER_GROUP_ID,CAST (avg(coalesce(TRAN_AMNT,0)) AS DECIMAL(20,2)) as TRAN_AMNT_MEAN,CAST (avg(coalesce(CBDT_INCOME_RANGE,0)) AS DECIMAL(20,2)) as CBDT_INCOME_RANGE_MEAN from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG  where PEER_GROUP_ID is not null AND peer_group_id not in {} group by PEER_GROUP_ID order by PEER_GROUP_ID ".format(peer_chg),conn)

In [96]:
cursor.execute('''UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG SET PEER_GROUP_ID = 'OTHERS_CHANGED' WHERE PEER_GROUP_ID IN {} '''.format(peer_chg))
conn.commit()

In [97]:
df = pd.read_sql_query('''select * from FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG with (nolock) where PEER_GROUP_ID = 'OTHERS_CHANGED' ''',conn)

In [98]:
df

,PERSON_MASTER_ID,PEER_GROUP_ID,PAN,DIN,CKYC_NUMBER,FULL_NAME,FATHER_NAME,MOTHER_NAME,DATE_OF_BIRTH,GENDER,CBDT_INCOME_RANGE,TRAN_AMNT_CREDIT,TRAN_AMNT_DEBIT,PEER_GROUP_AMNT,PINCODE,LATITUDE,LONGITUDE,OCCUPATION_BUSINESS,ACCOUNT_COUNT,ADDRESS_COUNT,MOBILE_COUNT,TELEPHONE_COUNT,EMAIL_COUNT,WATCHLIST_COUNT,LEA_REQUESTS_COUNT,LNK_REPORT_COUNT,STR_REPORT_COUNT,STR_AMOUNT_DEBIT,STR_AMOUNT_CREDIT,CTR_REPORT_COUNT,CTR_AMOUNT_DEBIT,CTR_AMOUNT_CREDIT,NTR_REPORT_COUNT,NTR_AMOUNT_DEBIT,NTR_AMOUNT_CREDIT,PTR_REPORT_COUNT,PTR_AMOUNT_DEBIT,PTR_AMOUNT_CREDIT,CBWTR_REPORT_COUNT,CBWTR_AMOUNT_DEBIT,CBWTR_AMOUNT_CREDIT,CCR_REPORT_COUNT,CCR_AMOUNT_DEBIT,CCR_AMOUNT_CREDIT,CREATE_DATE,UPDATE_DATE
0,6,OTHERS_CHANGED,ACDPI6767U,None,65700000000000,Manoj Kumar Tripathi,Harish Tripathi,None,1981-09-15,Male,None,nan,nan,0.00000,201306.00000,None,None,Self Employed,None,None,1.00000,nan,1.00000,None,None,None,1.00000,15000000.00000,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None
1,7,OTHERS_CHANGED,BCDPI8788I,None,None,Monica Tripathi,Nitin Tiwari,None,1990-08-20,Female,None,nan,15000000.00000,15000000.00000,201306.00000,None,None,Self Employed,None,None,1.00000,nan,1.00000,None,None,None,1.00000,15000000.00000,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None
2,43,OTHERS_CHANGED,FNHPK5558K,54814125,None,Ranjan S Rupesh,Karamver,None,1985-11-07,Male,None,1034801000000.00000,nan,1034801000000.00000,560100.00000,None,None,Government Sector,None,None,1.00000,nan,2.00000,None,None,None,2.00000,nan,75401000000.00000,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None
3,44,OTHERS_CHANGED,FJNPK5558K,None,None,Ramesh J Rajnikan,Kaluram,None,1979-10-23,Male,None,1034800000000.00000,nan,1034800000000.00000,560100.00000,None,None,Government Sector,None,None,1.00000,nan,1.00000,None,None,None,1.00000,nan,73600000000.00000,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None
4,48,OTHERS_CHANGED,ABNPI0954B,None,None,Qrithi W Keehin,kales,None,1991-04-17,Male,None,775997555028.00000,nan,775997555028.00000,560103.00000,None,None,Others,None,None,1.00000,nan,nan,None,None,None,nan,nan,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,419,OTHERS_CHANGED,DGFPY5814J,12545255,None,Sudarshan S Dake,None,None,1999-10-15,Male,None,nan,10000000.00000,10000000.00000,560100.00000,None,None,Others,None,None,1.00000,nan,nan,None,None,None,nan,nan,nan,nan,None,nan,1.00000,110000000.00000,nan,None,None,None,nan,None,None,None,None,None,None,None
76,420,OTHERS_CHANGED,DGFPY5815J,12545256,None,Varun G Dutt,None,None,1999-10-16,Male,None,nan,10000000.00000,10000000.00000,560100.00000,None,None,Self Employed,None,None,1.00000,nan,nan,None,None,None,nan,nan,nan,nan,None,nan,1.00000,110000000.00000,nan,None,None,None,nan,None,None,None,None,None,None,None
77,421,OTHERS_CHANGED,DGFPY5816J,12545257,None,Dalveer K Singh,None,None,1999-10-17,Male,None,nan,10000000.00000,10000000.00000,560100.00000,None,None,Others,None,None,1.00000,nan,nan,None,None,None,nan,nan,nan,nan,None,nan,1.00000,110000000.00000,nan,None,None,None,nan,None,None,None,None,None,None,None
78,422,OTHERS_CHANGED,DGFPY5817J,12545258,None,Eshita L Chatterjee,None,None,1999-10-18,Female,None,nan,10000000.00000,10000000.00000,560100.00000,None,None,Self Employed,None,None,1.00000,nan,nan,None,None,None,nan,nan,nan,nan,None,nan,1.00000,110000000.00000,nan,None,None,None,nan,None,None,None,None,None,None,None


In [99]:
# df.loc[0,['CBDT_INCOME_RANGE']] = [100000000000]

In [100]:
bins = [0,1,500000,1000000,2000000,3000000,4000000,5000000,7500000,10000000]
labels = ['OTHERS_CHANGED_0','OTHERS_CHANGED_1','OTHERS_CHANGED_2','OTHERS_CHANGED_3','OTHERS_CHANGED_4','OTHERS_CHANGED_5','OTHERS_CHANGED_6','OTHERS_CHANGED_7','OTHERS_CHANGED_8']
cluster_labels = [0,1,2,3,4,5,6,7,8]

In [101]:
df['PEER_GROUP_ID'] = pd.cut(x = df['PEER_GROUP_AMNT'], bins = bins, labels = labels, include_lowest = True)
df['Cluster_labels'] = pd.cut(x = df['PEER_GROUP_AMNT'], bins = bins, labels = cluster_labels, include_lowest = True)

In [102]:
df['PEER_GROUP_ID'] = df['PEER_GROUP_ID'].astype('str')

In [103]:
df['Cluster_labels'] = df['Cluster_labels'].astype(object)

In [104]:
df['PEER_GROUP_ID'].replace('nan','OTHERS_CHANGED_9',inplace =  True)


In [105]:
df['Cluster_labels'].fillna(9,inplace = True)
df['Cluster_labels'] = df['Cluster_labels'].astype('int32')

In [106]:
df.head()

,PERSON_MASTER_ID,PEER_GROUP_ID,PAN,DIN,CKYC_NUMBER,FULL_NAME,FATHER_NAME,MOTHER_NAME,DATE_OF_BIRTH,GENDER,CBDT_INCOME_RANGE,TRAN_AMNT_CREDIT,TRAN_AMNT_DEBIT,PEER_GROUP_AMNT,PINCODE,LATITUDE,LONGITUDE,OCCUPATION_BUSINESS,ACCOUNT_COUNT,ADDRESS_COUNT,MOBILE_COUNT,TELEPHONE_COUNT,EMAIL_COUNT,WATCHLIST_COUNT,LEA_REQUESTS_COUNT,LNK_REPORT_COUNT,STR_REPORT_COUNT,STR_AMOUNT_DEBIT,STR_AMOUNT_CREDIT,CTR_REPORT_COUNT,CTR_AMOUNT_DEBIT,CTR_AMOUNT_CREDIT,NTR_REPORT_COUNT,NTR_AMOUNT_DEBIT,NTR_AMOUNT_CREDIT,PTR_REPORT_COUNT,PTR_AMOUNT_DEBIT,PTR_AMOUNT_CREDIT,CBWTR_REPORT_COUNT,CBWTR_AMOUNT_DEBIT,CBWTR_AMOUNT_CREDIT,CCR_REPORT_COUNT,CCR_AMOUNT_DEBIT,CCR_AMOUNT_CREDIT,CREATE_DATE,UPDATE_DATE,Cluster_labels
0,6,OTHERS_CHANGED_0,ACDPI6767U,None,65700000000000,Manoj Kumar Tripathi,Harish Tripathi,None,1981-09-15,Male,None,nan,nan,0.00000,201306.00000,None,None,Self Employed,None,None,1.00000,nan,1.00000,None,None,None,1.00000,15000000.00000,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,0
1,7,OTHERS_CHANGED_9,BCDPI8788I,None,None,Monica Tripathi,Nitin Tiwari,None,1990-08-20,Female,None,nan,15000000.00000,15000000.00000,201306.00000,None,None,Self Employed,None,None,1.00000,nan,1.00000,None,None,None,1.00000,15000000.00000,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,9
2,43,OTHERS_CHANGED_9,FNHPK5558K,54814125,None,Ranjan S Rupesh,Karamver,None,1985-11-07,Male,None,1034801000000.00000,nan,1034801000000.00000,560100.00000,None,None,Government Sector,None,None,1.00000,nan,2.00000,None,None,None,2.00000,nan,75401000000.00000,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,9
3,44,OTHERS_CHANGED_9,FJNPK5558K,None,None,Ramesh J Rajnikan,Kaluram,None,1979-10-23,Male,None,1034800000000.00000,nan,1034800000000.00000,560100.00000,None,None,Government Sector,None,None,1.00000,nan,1.00000,None,None,None,1.00000,nan,73600000000.00000,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,9
4,48,OTHERS_CHANGED_9,ABNPI0954B,None,None,Qrithi W Keehin,kales,None,1991-04-17,Male,None,775997555028.00000,nan,775997555028.00000,560103.00000,None,None,Others,None,None,1.00000,nan,nan,None,None,None,nan,nan,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,9


In [107]:
df['OCCUPATION_BUSINESS_CLEAN'] = 'OTHERS_CHANGED'

In [108]:
df.head()

,PERSON_MASTER_ID,PEER_GROUP_ID,PAN,DIN,CKYC_NUMBER,FULL_NAME,FATHER_NAME,MOTHER_NAME,DATE_OF_BIRTH,GENDER,CBDT_INCOME_RANGE,TRAN_AMNT_CREDIT,TRAN_AMNT_DEBIT,PEER_GROUP_AMNT,PINCODE,LATITUDE,LONGITUDE,OCCUPATION_BUSINESS,ACCOUNT_COUNT,ADDRESS_COUNT,MOBILE_COUNT,TELEPHONE_COUNT,EMAIL_COUNT,WATCHLIST_COUNT,LEA_REQUESTS_COUNT,LNK_REPORT_COUNT,STR_REPORT_COUNT,STR_AMOUNT_DEBIT,STR_AMOUNT_CREDIT,CTR_REPORT_COUNT,CTR_AMOUNT_DEBIT,CTR_AMOUNT_CREDIT,NTR_REPORT_COUNT,NTR_AMOUNT_DEBIT,NTR_AMOUNT_CREDIT,PTR_REPORT_COUNT,PTR_AMOUNT_DEBIT,PTR_AMOUNT_CREDIT,CBWTR_REPORT_COUNT,CBWTR_AMOUNT_DEBIT,CBWTR_AMOUNT_CREDIT,CCR_REPORT_COUNT,CCR_AMOUNT_DEBIT,CCR_AMOUNT_CREDIT,CREATE_DATE,UPDATE_DATE,Cluster_labels,OCCUPATION_BUSINESS_CLEAN
0,6,OTHERS_CHANGED_0,ACDPI6767U,None,65700000000000,Manoj Kumar Tripathi,Harish Tripathi,None,1981-09-15,Male,None,nan,nan,0.00000,201306.00000,None,None,Self Employed,None,None,1.00000,nan,1.00000,None,None,None,1.00000,15000000.00000,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,0,OTHERS_CHANGED
1,7,OTHERS_CHANGED_9,BCDPI8788I,None,None,Monica Tripathi,Nitin Tiwari,None,1990-08-20,Female,None,nan,15000000.00000,15000000.00000,201306.00000,None,None,Self Employed,None,None,1.00000,nan,1.00000,None,None,None,1.00000,15000000.00000,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,9,OTHERS_CHANGED
2,43,OTHERS_CHANGED_9,FNHPK5558K,54814125,None,Ranjan S Rupesh,Karamver,None,1985-11-07,Male,None,1034801000000.00000,nan,1034801000000.00000,560100.00000,None,None,Government Sector,None,None,1.00000,nan,2.00000,None,None,None,2.00000,nan,75401000000.00000,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,9,OTHERS_CHANGED
3,44,OTHERS_CHANGED_9,FJNPK5558K,None,None,Ramesh J Rajnikan,Kaluram,None,1979-10-23,Male,None,1034800000000.00000,nan,1034800000000.00000,560100.00000,None,None,Government Sector,None,None,1.00000,nan,1.00000,None,None,None,1.00000,nan,73600000000.00000,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,9,OTHERS_CHANGED
4,48,OTHERS_CHANGED_9,ABNPI0954B,None,None,Qrithi W Keehin,kales,None,1991-04-17,Male,None,775997555028.00000,nan,775997555028.00000,560103.00000,None,None,Others,None,None,1.00000,nan,nan,None,None,None,nan,nan,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None,9,OTHERS_CHANGED


In [109]:
df = df[['PERSON_MASTER_ID','PEER_GROUP_ID','Cluster_labels','OCCUPATION_BUSINESS_CLEAN']]

In [110]:
for ind,row in df.iterrows():
#     print(row['PERSON_MASTER_ID'])
    cursor.execute("""Update fincore_db.fincore_analytics.risk_person_summary_stg set peer_group_id ='{}' where person_master_id = {} """.format(row['PEER_GROUP_ID'],row['PERSON_MASTER_ID']))
    conn.commit()
conn.close()

In [111]:
ind_data_U = ind_data_U.merge(df,how = 'left',on = 'PERSON_MASTER_ID')

In [112]:
df.head()

,PERSON_MASTER_ID,PEER_GROUP_ID,Cluster_labels,OCCUPATION_BUSINESS_CLEAN
0,6,OTHERS_CHANGED_0,0,OTHERS_CHANGED
1,7,OTHERS_CHANGED_9,9,OTHERS_CHANGED
2,43,OTHERS_CHANGED_9,9,OTHERS_CHANGED
3,44,OTHERS_CHANGED_9,9,OTHERS_CHANGED
4,48,OTHERS_CHANGED_9,9,OTHERS_CHANGED


In [113]:
ind_data_U['PEER_GROUP_ID_y'].fillna(value = ind_data_U['PEER_GROUP_ID_x'],inplace = True)

In [114]:
ind_data_U['Cluster_labels_y'].fillna(value = ind_data_U['Cluster_labels_x'],inplace = True)

In [115]:
ind_data_U['OCCUPATION_BUSINESS_CLEAN_y'].fillna(value = ind_data_U['OCCUPATION_BUSINESS_CLEAN_x'],inplace = True)

In [116]:
ind_data_U.drop(columns = ['PEER_GROUP_ID_x'],inplace = True)

In [117]:
ind_data_U.drop(columns = ['Cluster_labels_x'],inplace = True)

In [118]:
ind_data_U.drop(columns = ['OCCUPATION_BUSINESS_CLEAN_x'],inplace = True)

In [119]:
ind_data_U.rename(columns = {'PEER_GROUP_ID_y':'PEER_GROUP_ID'}, inplace = True)

In [120]:
ind_data_U.rename(columns = {'Cluster_labels_y':'Cluster_labels'}, inplace = True)

In [121]:
ind_data_U.rename(columns = {'OCCUPATION_BUSINESS_CLEAN_y':'OCCUPATION_BUSINESS_CLEAN'}, inplace = True)

In [122]:
# ind_data_U[ind_data_U['PERSON_MASTER_ID'] == 15046 ]

In [123]:
# BELOW IND_DATA_U NEEDS TO GO IN DATABASE 

In [124]:
ind_data_U.to_csv(r"/data/peer_updated.csv",index=False)

In [125]:
# from tqdm import tqdm

In [126]:
# for indO,rowO in tqdm(ind_data_U.iterrows()):
#     for ind,row in tqdm(df.iterrows()):
#         if (rowO['PERSON_MASTER_ID'] == row['PERSON_MASTER_ID']):
# #             print(rowO['PERSON_MASTER_ID'],rowO['PEER_GROUP_ID'],row['PEER_GROUP_ID'])
#             ind_data_U['PEER_GROUP_ID'][indO] = df['PEER_GROUP_ID'][ind]
#             ind_data_U['OCCUPATION_BUSINESS_CLEAN'][indO] = 'OTHERS_CHANGED'
#             break
    
        
    

In [127]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [128]:
RISK_LKP_PEER_GRP_ATTRIBUTES = '''

insert into FINCORE_DB.Fincore_Analytics.RISK_LKP_PEER_GRP_ATTRIBUTES
select PEER_GROUP_ID,
'I' AS ENTITY_TYPE,
COUNT(PERSON_MASTER_ID) AS RCD_CNT,
MAX(CBDT_INCOME_RANGE) as CBDT_INCOME_RANGE_MAX,
MIN(CBDT_INCOME_RANGE) as CBDT_INCOME_RANGE_MIN,

MAX(STR_AMOUNT_DEBIT)  as  STR_AMOUNT_DEBIT_MAX,
MIN(STR_AMOUNT_DEBIT)  as  STR_AMOUNT_DEBIT_MIN,
MAX(STR_AMOUNT_CREDIT)  as STR_AMOUNT_CREDIT_MAX,
MIN(STR_AMOUNT_CREDIT)  as STR_AMOUNT_CREDIT_MIN,

MAX(CTR_AMOUNT_DEBIT)  as  CTR_AMOUNT_DEBIT_MAX,
MIN(CTR_AMOUNT_DEBIT)  as  CTR_AMOUNT_DEBIT_MIN,
MAX(CTR_AMOUNT_CREDIT)  as CTR_AMOUNT_CREDIT_MAX,
MIN(CTR_AMOUNT_CREDIT)  as CTR_AMOUNT_CREDIT_MIN,

MAX(NTR_AMOUNT_DEBIT)  as  NTR_AMOUNT_DEBIT_MAX,
MIN(NTR_AMOUNT_DEBIT)  as  NTR_AMOUNT_DEBIT_MIN,
MAX(NTR_AMOUNT_CREDIT)  as NTR_AMOUNT_CREDIT_MAX,
MIN(NTR_AMOUNT_CREDIT)  as NTR_AMOUNT_CREDIT_MIN,

MAX(PTR_AMOUNT_DEBIT)  as  PTR_AMOUNT_DEBIT_MAX,
MIN(PTR_AMOUNT_DEBIT)  as  PTR_AMOUNT_DEBIT_MIN,
MAX(PTR_AMOUNT_CREDIT)  as PTR_AMOUNT_CREDIT_MAX,
MIN(PTR_AMOUNT_CREDIT)  as PTR_AMOUNT_CREDIT_MIN,

MAX(CBWTR_AMOUNT_DEBIT)  as  CBWTR_AMOUNT_DEBIT_MAX,
MIN(CBWTR_AMOUNT_DEBIT)  as  CBWTR_AMOUNT_DEBIT_MIN,
MAX(CBWTR_AMOUNT_CREDIT)  as CBWTR_AMOUNT_CREDIT_MAX,
MIN(CBWTR_AMOUNT_CREDIT)  as CBWTR_AMOUNT_CREDIT_MIN,

MAX(CCR_AMOUNT_DEBIT)  as  CCR_AMOUNT_DEBIT_MAX,
MIN(CCR_AMOUNT_DEBIT)  as  CCR_AMOUNT_DEBIT_MIN,
MAX(CCR_AMOUNT_CREDIT)  as CCR_AMOUNT_CREDIT_MAX,
MIN(CCR_AMOUNT_CREDIT)  as CCR_AMOUNT_CREDIT_MIN,
GETDATE()

from FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG GROUP BY PEER_GROUP_ID  
'''


cursor.execute(RISK_LKP_PEER_GRP_ATTRIBUTES)
conn.commit()



In [129]:
conn.close()

In [130]:
# peer_grp_1.head()

In [131]:
ind_data_U['Cluster_labels'] = ind_data_U['Cluster_labels'].astype('int32')

In [132]:
ind_data_U.to_csv(r"/data/peer_updated.csv",index=False)

In [133]:
# for ind,row in ind_data_U_temp.iterrows():
#     occ = row[1]
#     print(occ)


#     d = {}
#     try:
#     #     print(row['PERSON_MASTER_ID'])
#         for indP,rowP in peer_grp_1.iterrows():
#             if rowP[0].startswith(occ):
#                 dis = math.sqrt(((rowP['TRAN_AMNT_MEAN'] - row['TRAN_AMNT'])**2)+((rowP['CBDT_INCOME_RANGE_MEAN'] - row['CBDT_INCOME_RANGE'])**2))
#                 d[rowP[0]] = dis


# #         cursor.execute("""update fincore_analytics.RISK_PERSON_SUMMARY_STG 
# #         set RISK_PERSON_SUMMARY_STG.PEER_GROUP_ID = '{}'
# #         where RISK_PERSON_SUMMARY_STG.PERSON_MASTER_ID = {}""".format( min(d), row['PERSON_MASTER_ID']))  
#         ind_data_U_temp['Peer_group_changed'] = min(d)
#     except ValueError:
# #         cursor.execute("""update fincore_analytics.RISK_PERSON_SUMMARY_STG 
# #         set RISK_PERSON_SUMMARY_STG.PEER_GROUP_ID = '{}'
# #         where RISK_PERSON_SUMMARY_STG.PERSON_MASTER_ID = {}""".format(occ, row['PERSON_MASTER_ID'])) 
#         ind_data_U_temp['PEER_GROUP_ID_changed'] = occ
        

# # # conn.commit()
# # # conn.close()
            
        
    

In [134]:
# for ind,row in ind_data_U_temp.iterrows():
# #     print(row['PERSON_MASTER_ID'])
#     occ = row[1]
#     print("occ",occ)
# #     break

#     d = {}

#     try:
#     #     print(row['PERSON_MASTER_ID'])
#         for indP,rowP in peer_grp_1.iterrows():
# #             print(rowP[0])
#             if rowP[0].startswith(occ):
#                 dis = math.sqrt(((rowP['TRAN_AMNT_MEAN'] - row['TRAN_AMNT'])**2)+((rowP['CBDT_INCOME_RANGE_MEAN'] - row['CBDT_INCOME_RANGE'])**2))
#                 d[rowP[0]] = dis
                
                
        
            
# #         cursor.execute("""update fincore_analytics.RISK_PERSON_SUMMARY_STG 
# #         set RISK_PERSON_SUMMARY_STG.PEER_GROUP_ID = '{}'
# #         where RISK_PERSON_SUMMARY_STG.PERSON_MASTER_ID = {}""".format( min(d), row['PERSON_MASTER_ID']))  
#         Peer_group_changed = min(d)
    
#     except ValueError:
# #         cursor.execute("""update fincore_analytics.RISK_PERSON_SUMMARY_STG 
# #         set RISK_PERSON_SUMMARY_STG.PEER_GROUP_ID = '{}'
# #         where RISK_PERSON_SUMMARY_STG.PERSON_MASTER_ID = {}""".format(occ, row['PERSON_MASTER_ID'])) 
#         Peer_group_changed = occ
    
    
#     print(Peer_group_changed)
    
#     ind_data_U_temp['Peer_group_changed'][ind] = Peer_group_changed
        
     

In [135]:
# ind_data_U_temp

In [136]:
# ind_data_U_temp.to_csv('results1.csv')